In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import pandas as pd
import json

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
data = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE");

In [ ]:
for x in range(1,7):
    s_date = '2018-' + str(x) + '-1'
    e_date = '2018-' + str(x) + '-28'
    
    dataset = data.filterDate(s_date, e_date)

    total = dataset.reduce(ee.Reducer.sum());

    ranchi = gpd.read_file("./ranchi/ranchi_new.shp")

    js = json.loads(ranchi.to_json())
    shpz = ee.Geometry(ee.FeatureCollection(js).geometry())

    stats = total.reduceRegion(
      reducer= ee.Reducer.mean(),
      geometry= shpz,
      scale= 5000,
      )

    print("Amount = ",stats.get('pr_sum').getInfo())

    count = total.reduceRegion(
      reducer= ee.Reducer.count(),
      geometry= shpz,
      scale= 5000,
      )

    print("Total Pixel = ", count.get('pr_sum').getInfo())

    totalRanchi = total.clip(shpz)


    vectors = totalRanchi.sampleRegions(
      collection=shpz,
      geometries= True,
      scale= 5000,
    )
    
    veca = 'vec_'+str(x)
    print(veca)
    
    task_config = {
      'collection': vectors,
      'folder': 'earthengine_all',
      'fileNamePrefix': veca,
      'fileFormat': 'CSV'}

    task=ee.batch.Export.table.toDrive(**task_config)
    task.start()